# 準備

## データの収集

In [ ]:
import google.colab.drive
google.colab.drive.mount('gdrive')
!ls ./gdrive

## Googleドライブからデータをコピー

In [ ]:
!mkdir -p 'gdrive/MyDrive/cyclegan'

In [ ]:
# photo2portraitデータセットのコピー
!gdown --id 1arF3guFms5tLiaIs8GtcV2dW0WAvrvLM

In [ ]:
!ls /content
!cp /content/photo2portrait.tar.gz .

In [ ]:
!tar -zxvf photo2portrait.tar.gz 

In [ ]:
import matplotlib.pyplot as plt

# ドメインAの学習データ
img = plt.imread('./photo2portrait/train/A/000177.jpg')
plt.imshow(img)

In [ ]:
# ドメインBの学習データ
img = plt.imread('./photo2portrait/train/B/000212.jpg')
plt.imshow(img)

## ライブラリのインストール

In [ ]:
!pip install torch>=0.4.1
!pip install torchvision>=0.2.1
!pip install dominate>=2.3.1
!pip install visdom>=0.1.8.
!pip install tensorboard
!pip install tensorboardX

## インポート

In [ ]:
import glob
import random
import os
import numpy as np
import time
import datetime
import sys

from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch
from visdom import Visdom

import itertools
from PIL import Image

In [ ]:
# ドメインAとドメインBの画像データセット生成クラス
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode='train'):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, '%s/A' % mode) + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, '%s/B' % mode) + '/*.*'))

    def __getitem__(self, index):
        item_A = self.transform(Image.open(self.files_A[index % len(self.files_A)]).convert('RGB'))

        if self.unaligned:
            item_B = self.transform(Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)]).convert('RGB'))
        else:
            item_B = self.transform(Image.open(self.files_B[index % len(self.files_B)]).convert('RGB'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

# ネットワーク定義

## Residual Block

In [ ]:
# ResidualBlockの定義
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.conv_block(x)

## 生成器

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),

            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),

            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, 7),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


## 識別器

In [ ]:

class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.InstanceNorm2d(128), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, padding=1),
            nn.InstanceNorm2d(512), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, x):
        x =  self.model(x)
        # Average pooling and flatten
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

## 生成画像のバッファ

In [ ]:
# 過去の生成データ(50iter分)を保持しておく
class ReplayBuffer():
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            #
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0,1) > 0.5:
                    i = random.randint(0, self.max_size-1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))


In [ ]:
class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

In [ ]:
!mkdir -p output
!ls output

# 学習

## TensorboardでのLoss確認

In [ ]:
from torch.utils.tensorboard import SummaryWriter
log_dir = './logs'
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
%load_ext tensorboard
%tensorboard --logdir './logs'

## パラメータ設定

In [ ]:

class Opts():
    def __init__(self):
        self.start_epoch = 0
        self.n_epochs = 5
        self.batch_size = 1
        self.dataroot = './photo2portrait/'
        self.lr = 0.0002
        self.decay_epoch = 200
        self.size = 256
        self.input_nc = 3
        self.output_nc = 3
        self.cpu = False
        self.n_cpu = 8
        self.device_name = "cuda:0"
        self.device = torch.device(self.device_name)
        self.load_weight = False

opt = Opts()


## ネットワーク呼び出し

In [ ]:

# 生成器
netG_A2B = Generator(opt.input_nc, opt.output_nc)
netG_B2A = Generator(opt.output_nc, opt.input_nc)

# 識別器
netD_A = Discriminator(opt.input_nc)
netD_B = Discriminator(opt.output_nc)

# GPU
if not opt.cpu:
    netG_A2B.cuda()
    netG_B2A.cuda()
    netD_A.cuda()
    netD_B.cuda()

# 重みパラメータ初期化
netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

# 保存したモデルのロード
if opt.load_weight is True:
    netG_A2B.load_state_dict(torch.load("./output/netG_A2B.pth", map_location="cuda:0"), strict=False)
    netG_B2A.load_state_dict(torch.load("./output/netG_B2A.pth", map_location="cuda:0"), strict=False)
    netD_A.load_state_dict(torch.load("./output/netD_A.pth", map_location="cuda:0"), strict=False)
    netD_B.load_state_dict(torch.load("./output/netD_B.pth", map_location="cuda:0"), strict=False)

# 損失関数
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

# Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                                lr=opt.lr, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=opt.lr, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=opt.lr, betas=(0.5, 0.999))

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(opt.n_epochs, opt.start_epoch, opt.decay_epoch).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(opt.n_epochs, opt.start_epoch, opt.decay_epoch).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(opt.n_epochs, opt.start_epoch, opt.decay_epoch).step)

# 入出力メモリ確保
Tensor = torch.cuda.FloatTensor if not opt.cpu else torch.Tensor
input_A = Tensor(opt.batch_size, opt.input_nc, opt.size, opt.size)
input_B = Tensor(opt.batch_size, opt.output_nc, opt.size, opt.size)
target_real = Variable(Tensor(opt.batch_size).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(opt.batch_size).fill_(0.0), requires_grad=False)

# 過去データ分のメモリ確保
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# データローダー
transforms_ = [ transforms.Resize(int(opt.size*1.12), Image.BICUBIC), 
                transforms.RandomCrop(opt.size), 
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(opt.dataroot, transforms_=transforms_, unaligned=True), 
                        batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu)

print("num dataloader= {}".format(len(dataloader)))

In [ ]:
import torchsummary
torchsummary.summary(netG_A2B, (opt.input_nc, opt.size, opt.size))


In [ ]:
torchsummary.summary(netD_A, (opt.input_nc, opt.size, opt.size))

In [ ]:
def save_loss(train_info, batches_done):
    """
    lossの保存
    """
    for k, v in train_info.items():
        writer.add_scalar(k, v, batches_done)


## 学習の開始

In [ ]:

for epoch in range(opt.start_epoch, opt.n_epochs):
    for i, batch in enumerate(dataloader):
        # モデルの入力
        real_A = Variable(input_A.copy_(batch['A']))
        real_B = Variable(input_B.copy_(batch['B']))

        ##### 生成器A2B、B2Aの処理 #####
        optimizer_G.zero_grad()

        # 同一性損失の計算（Identity loss)
        # G_A2B(B)はBと一致
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B)*5.0
        # G_B2A(A)はAと一致
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A)*5.0

        # 敵対的損失（GAN loss）
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)

        fake_A = netG_B2A(real_B)
        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, target_real)

        # サイクル一貫性損失（Cycle-consistency loss）
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*10.0

        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*10.0

        # 生成器の合計損失関数（Total loss）
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()
        
        optimizer_G.step()

        ##### ドメインAの識別器 #####
        optimizer_D_A.zero_grad()

        # ドメインAの本物画像の識別結果（Real loss）
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # ドメインAの生成画像の識別結果（Fake loss）
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # 識別器（ドメインA）の合計損失（Total loss）
        loss_D_A = (loss_D_real + loss_D_fake)*0.5
        loss_D_A.backward()

        optimizer_D_A.step()

        ##### ドメインBの識別器 #####
        optimizer_D_B.zero_grad()

        # ドメインBの本物画像の識別結果（Real loss）
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)
        
        # ドメインBの生成画像の識別結果（Fake loss）
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # 識別器（ドメインB）の合計損失（Total loss）
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D_B.step()
        ###################################

        if i % 20 == 0:
            print('Epoch[{}]({}/{}) loss_G: {:.4f} loss_G_identity: {:.4f} loss_G_GAN: {:.4f} loss_G_cycle: {:.4f} loss_D: {:.4f}'.format(
                epoch, i, len(dataloader), loss_G, (loss_identity_A + loss_identity_B),
                (loss_GAN_A2B + loss_GAN_B2A), (loss_cycle_ABA + loss_cycle_BAB), (loss_D_A + loss_D_B)
                ))

        train_info = {
            'epoch': epoch, 
            'batch_num': i, 
            'lossG': loss_G.item(),
            'lossG_identity': (loss_identity_A.item() + loss_identity_B.item()),
            'lossG_GAN': (loss_GAN_A2B.item() + loss_GAN_B2A.item()),
            'lossG_cycle': (loss_cycle_ABA.item() + loss_cycle_BAB.item()),
            'lossD': (loss_D_A.item() + loss_D_B.item()), 
            }

        batches_done = (epoch - 1) * len(dataloader) + i
        save_loss(train_info, batches_done)

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    # Save models checkpoints
    torch.save(netG_A2B.state_dict(), 'output/netG_A2B.pth')
    torch.save(netG_B2A.state_dict(), 'output/netG_B2A.pth')
    torch.save(netD_A.state_dict(), 'output/netD_A.pth')
    torch.save(netD_B.state_dict(), 'output/netD_B.pth')

In [ ]:
!ls output

# テスト

## パラメータ設定

In [ ]:

class Opts_test():
    def __init__(self):
        self.batch_size = 1
        self.dataroot = './photo2portrait/'
        self.size = 256
        self.input_nc = 3
        self.output_nc = 3
        self.cpu = False
        self.n_cpu = 8
        self.device_name = "cuda:0"
        self.device = torch.device(self.device_name)
        self.load_weight = False
        self.generator_A2B = 'output/netG_A2B.pth'
        self.generator_B2A = 'output/netG_B2A.pth'
        self.cuda = True

opt2 = Opts_test()


## ネットワーク呼び出し

In [ ]:

# 生成器G
netG_A2B = Generator(opt2.input_nc, opt2.output_nc)
netG_B2A = Generator(opt2.output_nc, opt2.input_nc)

# CUDA
if opt2.cuda:
    netG_A2B.cuda()
    netG_B2A.cuda()

# Load state dicts
netG_A2B.load_state_dict(torch.load(opt2.generator_A2B))
netG_B2A.load_state_dict(torch.load(opt2.generator_B2A))

# Set model's test mode
netG_A2B.eval()
netG_B2A.eval()

# Inputs & targets memory allocation
Tensor = torch.cuda.FloatTensor if opt2.cuda else torch.Tensor
input_A = Tensor(opt2.batch_size, opt2.input_nc, opt2.size, opt2.size)
input_B = Tensor(opt2.batch_size, opt2.output_nc, opt2.size, opt2.size)

# Dataset loader
transforms_ = [transforms.Resize(int(opt2.size*1.0), Image.BICUBIC), 
                transforms.RandomCrop(opt2.size), 
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(opt2.dataroot, transforms_=transforms_, mode='train'), 
                        batch_size=opt2.batch_size, shuffle=False, num_workers=opt2.n_cpu)


# 出力用フォルダ生成
if not os.path.exists('output/A'):
    os.makedirs('output/A')
if not os.path.exists('output/B'):
    os.makedirs('output/B')

## テスト実行

In [ ]:
##### 生成器Gによる画像生成　#####
from torchvision.utils import save_image

num_create = 100

for i, batch in enumerate(dataloader):
    # Set model input
    real_A = Variable(input_A.copy_(batch['A']))
    real_B = Variable(input_B.copy_(batch['B']))

    # Generate output
    fake_B = 0.5*(netG_A2B(real_A).data + 1.0)
    fake_A = 0.5*(netG_B2A(real_B).data + 1.0)

    out_img1 = torch.cat([real_A, fake_B], dim=2)
    out_img2 = torch.cat([real_B, fake_A], dim=2)

    # Save image files
    save_image(out_img1, 'output/A/%04d.png' % (i+1))
    save_image(out_img2, 'output/B/%04d.png' % (i+1))

    if i > num_create:
        break


## 生成画像の表示

In [ ]:
# A->Bの生成画像
import matplotlib.pyplot as plt
img = plt.imread('output/A/0001.png')
plt.imshow(img)

In [ ]:
# B->Aの生成画像
img = plt.imread('output/B/0001.png')
plt.imshow(img)

# 結果の取得

## 生成画像をローカルにコピー

In [ ]:
from google.colab import files
!tar -zcvf outputA.tar.gz output/A/*
!tar -zcvf outputB.tar.gz output/B/*

!cp "outputA.tar.gz" 'gdrive/MyDrive/cyclegan/'
!cp "outputB.tar.gz" 'gdrive/MyDrive/cyclegan/'

## モデルの保存

In [ ]:
!ls output

In [ ]:
!cp output/net*.pth 'gdrive/MyDrive/cyclegan/'

In [ ]:
!ls 'gdrive/MyDrive/cyclegan/'